In [1]:
#SE IMPORTAN LAS LIBRERÍAS NECESARIAS.
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#SE IMPORTA EL ARCHIVO DE DATOS QUE SE EXPORTÓ DESDE R.
datos=pd.read_csv('chalets_baratos.csv',sep=',',quotechar='"',index_col=0)
datos.head(5)

,est_sqft_liv_area,est_valor_zipcode,est_num_beds,est_num_baths,est_year_built,pool_yn.Yes,est_garage_spaces,sale_price
1,-1.158703,-0.088449,-1.684392,-1.552425,-0.789076,0,-1.351284,310000.0
2,0.009971,0.077384,-0.562267,0.661896,-0.175190,0,0.503854,380000.0
3,-0.445946,0.077384,0.559857,-0.445265,1.052582,0,0.503854,235000.0
4,-1.184490,0.077384,-0.562267,-0.445265,-0.930742,0,1.431423,264997.0
5,-0.358270,0.077384,-0.562267,-0.445265,-0.930742,1,-1.351284,410000.0


In [3]:
#SE ASIGNAN ROLES A LAS VARIABLES.
X,y=datos.iloc[:,0:7],datos.iloc[:,-1]

In [4]:
#SE COMPRUEBA QUE LOS DATOS ESTÁN BIEN SEPARADOS.
X.head(5)

,est_sqft_liv_area,est_valor_zipcode,est_num_beds,est_num_baths,est_year_built,pool_yn.Yes,est_garage_spaces
1,-1.158703,-0.088449,-1.684392,-1.552425,-0.789076,0,-1.351284
2,0.009971,0.077384,-0.562267,0.661896,-0.175190,0,0.503854
3,-0.445946,0.077384,0.559857,-0.445265,1.052582,0,0.503854
4,-1.184490,0.077384,-0.562267,-0.445265,-0.930742,0,1.431423
5,-0.358270,0.077384,-0.562267,-0.445265,-0.930742,1,-1.351284


In [5]:
#SE DIVIDEN LOS DATOS. 75% TRAIN Y 25% TEST. ESTO, PARA APLICAR LA FUNCIÓN PREDICT PARA DATOS NUEVOS.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=21045)

In [6]:
#MODELO XGBOOST CON HIPERPARÁMETROS DEFINIDOS EN R Y VALIDACIÓN CRUZADA.
modelo=xgb.XGBRegressor(min_child_weight=5,n_estimators=800,max_depth=3,learning_rate=0.1,gamma=0.01,subsample=0.95,
                         colsample_bytree=0.8,reg_alpha=0.01,reg_lambda=0.1,base_score=0.123,objective='reg:linear')
kfold=KFold(n_splits=10,random_state=70523)
results=cross_val_score(modelo,X_train,y_train,cv=kfold)
print("R-Cuadrado: %.2f" % (results.mean()))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[11:14:29] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[11:14:31] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[11:14:31] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[11:14:34] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[11:14:34] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[11:14:36] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[11:14:36] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is no

Este R2 es irrelevante, pues los datos tienen muy poca varianza. Para obtener un R2 representativo, se ha de calcular con todos los chalets, baratos y caros, calculando el SCT sobre la media de precios de todos los chalets.

In [7]:
#GENERACIÓN DEL MODELO.
modelo.fit(X,y)

[11:14:53] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[11:14:56] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.123, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.01, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=3,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=800, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0.01,
             reg_lambda=0.1, scale_pos_weight=1, subsample=0.95,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
#PREDICCIONES SOBRE 20% DE DATOS TEST SEPARADOS.
y_pred=modelo.predict(X_test)
predictions=[round(value) for value in y_pred]

In [9]:
print('R Cuadrado en datos test: %.2f' % r2_score(y_test,predictions))

R Cuadrado en datos test: 0.79


In [12]:
import joblib
filename = 'modelo_baratos_chalets.model'
joblib.dump(modelo,filename)

['modelo_baratos_chalets.model']